In [1]:
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from music_db_finder.fuzz_helpers import obj_ratio, get_name_as_str
import time

import dotenv
env_file_name = '.env'
dotenv.load_dotenv(env_file_name, override=True)

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')
YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)

from ytmusicapi import YTMusic as YTMusicClient
import json

with open('../../oauth.json__') as file:
    oauth = json.load(file)
oauth['access_token'] = YTMUSIC_ACCESS_TOKEN
oauth['refresh_token'] = YTMUSIC_REFRESH_TOKEN
ytMusic = YTMusicClient(oauth)

In [2]:
os.getenv('SPOTIFY_CLIENT_ID__')

In [3]:
ytMusic.search('Oasis Wonderwall')

Exception: Server returned HTTP 400: Bad Request.
Request contains an invalid argument.

In [2]:
gr = ytMusic.search('grendel')
gr


Exception: Server returned HTTP 400: Bad Request.
Request contains an invalid argument.

In [2]:
# UNIVERSAL
def match_exact(search_string, results):
    candidate = None
    for result in results:
        res_lower = get_name_as_str(result)
        names = [res_lower.lower(), res_lower.lower().rsplit(' (', maxsplit=1)[0]]
        if search_string.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def match_fuzzy(search_string, results, scorer=obj_ratio):
    kwargs = {'scorer':scorer, 'limit':None, 'score_cutoff':70}
    fuzzy_results = r_process.extract(search_string, results, **kwargs)
    return [res[0] for res in fuzzy_results]



In [3]:
# DISCOGS
def find_discogs_exact(search_string, search_type):
    results = discogs_client.search(search_string, type=search_type)
    return match_exact(search_string, results)

def find_discogs_fuzzy(search_string, search_type, scorer=obj_ratio):
    results = discogs_client.search(search_string, type=search_type)
    return match_fuzzy(search_string, results, scorer=scorer)

def find_discogs_band_exact(band_name):
    """Returns the artist object for searches that have only one exact match, 
    ignoring case. If multiple results are returned, returns None.
    
    It works by iterating through ALL the results and comparing search string to name
    and truncated name (one split from the right, for (1) duplicated names). If it
    finds two of the same string it exists with None.
    """
    # DONE optimization: iterate over results until two are found
    # if only 1 exists, it goes through it all, as it does now
    # if 2+ exist, it goes until it founds second
    # TODO optimization: use as chained, returning status and result
    # if only 1 exist return true and the object as a list
    # if multiple exists return false and all candidates as a list
    ### Those are contradicting functionalities, enable a switch?
    # Not required - the first one if generally faster:
    # If you want to get just the artist and it exist - you need to iterate all.
    # If you want to get just the artist and multiple exist:
    #   you have to iterate again through the next function, through all results
    #   or you give up
    # For first to be slower it needs to iterate through the list twice:
    #   that is, the second same artist must be at the very end
    #   and we can rely on search optimization that they push similar artists ahead
    # DONE: check if you've iterated through them all - for _ in result iterates all
    band_results = discogs_client.search(band_name, type='artist')
    candidate = None
    for result in band_results:
        names = [(res_lower := result.name.lower()), res_lower.rsplit(maxsplit=1)[0]]
        if band_name.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def discogs_exact_and_fuzzy_search_band_to_album(band_name, album_name):
    # 1 - find bend exact
    result_band = find_discogs_exact(band_name, 'artist')
    if result_band:
        # 2 - find album exact
        releases = result_band.releases
        result_album = match_exact(album_name, releases)
        if result_album:
            print('found exact both')
            return [(result_band, [result_album])]
        else:
            # 3 - if not exact, find fuzzy
            candidates_album = match_fuzzy(album_name, releases)
            print('found exact band, fuzzy albumns')
            return [(result_band, candidates_album)]

    # 4 - if not find fuzzy 
    else:
        results_band = find_discogs_fuzzy(band_name, 'artist')
        # 5 - iterate through result and break on first exactž
        results = []
        candidates_album = []
        count = 0
        for band in results_band:
            try:
                releases = band.releases
            except Exception:
                continue
            result = match_exact(album_name, releases)
            if result:
                print('fuzzy band, exact album')
                return [(band, [result])]
            result = match_fuzzy(album_name, releases)
            if result:
                candidates_album.extend(result)
                results.append((band, result))
                count = 0
            else:
                count += 1
                if count == 5:
                    break
        print('fuzzy band, fuzzy album')
        return results


In [ ]:
# SPOTIFY

def find_spotify_exact(search_string, search_type):
    results = spotify_client.search(search_string, type=search_type, limit=50)[search_type + 's']['items']
    return match_exact(search_string, results)

def find_spotify_fuzzy(search_string, search_type, scorer=obj_ratio):
    results = spotify_client.search(search_string, type=search_type, limit=50)[search_type + 's']['items']
    return match_fuzzy(search_string, results, scorer=scorer)

def spotify_exact_and_fuzzy_search_band_to_album(band_name, album_name):
    # 1 - find bend exact
    result_band = find_spotify_exact(band_name, 'artist')
    if result_band:
        # 2 - find album exact
        # releases = result_band.releases
        releases = spotify_client.artist_albums(result_band['id'])['items']
        result_album = match_exact(album_name, releases)
        if result_album:
            print('found exact both')
            return [(result_band, [result_album])]
        else:
            # 3 - if not exact, find fuzzy
            candidates_album = match_fuzzy(album_name, releases)
            print('found exact band, fuzzy albumns')
            return [(result_band, candidates_album)]

    # 4 - if not find fuzzy 
    else:
        results_band = find_spotify_fuzzy(band_name, 'artist')
        # 5 - iterate through result and break on first exactž
        results = []
        candidates_album = []
        count = 0
        for band in results_band:
            print(band['name'])
            try:
                releases = spotify_client.artist_albums(band['id'])['items']
            except Exception:
                continue
            result = match_exact(album_name, releases)
            if result:
                print('fuzzy band, exact album')
                return [(band, [result])]
                break
            result = match_fuzzy(album_name, releases)
            if result:
                candidates_album.extend(result)
                results.append((band, result))
                count = 0
            else:
                count += 1
                if count == 5:
                    break
        print('fuzzy band, fuzzy album')
        return results


In [5]:
# YTMUSIC
def find_ytmusic_exact(search_string, search_type):
    results = ytMusic.search(search_string, filter=search_type)
    return match_exact(search_string, results)

def find_ytmusic_fuzzy(search_string, search_type, scorer=obj_ratio):
    results = ytMusic.search(search_string, filter=search_type)
    return match_fuzzy(search_string, results, scorer=scorer)

def ytmusic_exact_and_fuzzy_search_band_to_album(band_name, album_name):
    # 1 - find bend exact
    result_band = find_ytmusic_exact(band_name, 'artists')
    if result_band:
        # 2 - find album exact
        result_id = result_band['browseId']
        releases = ytMusic.get_artist(result_id)['albums']['results']
        result_album = match_exact(album_name, releases)
        if result_album:
            print('found exact both')
            return [(result_band, [result_album])]
        else:
            # 3 - if not exact, find fuzzy
            candidates_album = match_fuzzy(album_name, releases)
            print('found exact band, fuzzy albumns')
            return [(result_band, candidates_album)]

    # 4 - if not find fuzzy 
    else:
        results_band = find_ytmusic_fuzzy(band_name, 'artists')
        # 5 - iterate through result and break on first exactž
        results = []
        candidates_album = []
        count = 0
        for band in results_band:
            print(band['artist'])
            result_id = band['browseId']
            try:
                releases = ytMusic.get_artist(result_id)['albums']['results']
            except Exception:
                continue
            result = match_exact(album_name, releases)
            if result:
                print('fuzzy band, exact album')
                return [(band, [result])]
                break
            result = match_fuzzy(album_name, releases)
            if result:
                candidates_album.extend(result)
                results.append((band, result))
                count = 0
            else:
                count += 1
                if count == 5:
                    break
        print('fuzzy band, fuzzy album')
        return results

In [ ]:
print(find_band_exact('combichrist'))  # <Artist 139618 'Combichrist'>
print(find_band_exact('grendel'))  # None

print(len(list(find_band_fuzzy('combichrist'))))  # 1
print(len(list(find_band_fuzzy('grendel'))))  # 14

print(find_exact('combichrist', 'artist'))  # <Artist 139618 'Combichrist'>
print(find_exact('Today We Are All Demons', 'master'))  # <Artist 139618 'Combichrist'>
print(find_exact('Today We Are All Demons', 'release'))  # None
print(find_exact('floodland', 'master'))  # None

print(len(list(find_fuzzy('combichrist', 'artist'))))  # 1
print(len(list(find_fuzzy('Today We Are All Demons', 'master'))))  # 1
print(len(list(find_fuzzy('Today We Are All Demons', 'release'))))  # 12
print(len(list(find_fuzzy('Floodland', 'master'))))  # 2



In [46]:
result = find_spotify_fuzzy(band_name, 'artist')
[(res['name'], res['id']) for res in result]

[('Combichrist', '5556RAgwP70PnXbaUTGNbR'),
 ('Combichrist', '43cDNHhDAQabilqQk7SVEk')]

In [ ]:
band_name = 'welle:erdball'
search_string = band_name
search_type = 'artist'
results = spotify_client.search(search_string, type=search_type, limit=50)[search_type + 's']['items']
[(res['name'], res['external_urls'], res['id']) for res in results]
# [res['external_urls'] for res in results if res['name'].lower == 'x-rx']
# results[0]

In [47]:
result = spotify_client.artist_albums('5556RAgwP70PnXbaUTGNbR')['items']

